In [1]:
import pickle
from utils.analysis_from_interaction import *
from egg.core.language_analysis import Disent
from language_analysis_local import TopographicSimilarityConceptLevel, encode_target_concepts_for_topsim
from utils.analysis_tools_lazimpa import *

# calculate metrics from stored interactions

In [2]:
datasets = ('(3,4)', '(3,8)', '(3,16)', '(4,4)', '(4,8)', '(5,4)')
n_attributes = (3, 3, 3, 4, 4, 5)
n_values = (4, 8, 16, 4, 8, 4)
n_epochs = 300
#paths = ['results/' + d + '_sample_scaling_10_balanced_False_vsf_3/' for d in datasets]
paths = ['results/' + d + '_game_size_10_vsf_3/' for d in datasets]



In [3]:
datasets = ('(4,4)',)
n_attributes = (4,)
n_values = (4,)
n_epochs = 100
#paths = ['results/' + d + '_sample_scaling_10_balanced_False_vsf_3/' for d in datasets]
paths = ['results/' + d + '_game_size_10_vsf_3/' for d in datasets]

In [4]:
datasets = ['(3,4)',]
n_values = [4,]
n_attributes = [3,]
n_epochs = 300
paths = ['results/' + d + '_game_size_10_vsf_3' for d in datasets]

In [5]:
context_unaware = False # whether original or context_unaware simulations are evaluated
lazy = True # whether the lazy agent is used
impatient = False
setting = ""
if not lazy:
    start = 'impatience_' if impatient else ''
    if context_unaware:
        setting = start + 'context_unaware'
    else:
        setting = start + 'context_aware'
        if impatient == False:
            setting = 'standard'
else:
    start = 'lazimpa' if impatient else 'lazy'
    if context_unaware:
        setting = start + '_context_unaware'
    else:
        setting = start + '_context_aware'
setting

## entropy scores: MI, effectiveness, efficiency

In [6]:
for d in range(len(datasets)):
    
    for run in range(1): #5

        path_to_run = paths[d] + str(setting) +'/' + str(run) + '/' 
        interaction = load_interaction(paths[d],setting,run,n_epochs)

        attributes = n_attributes[d]
        values = n_values[d]
        scores = information_scores(interaction, attributes, values, normalizer="arithmetic")

        #pickle.dump(scores, open(path_to_run + 'entropy_scores.pkl', 'wb'))
scores

results/(3,4)_game_size_10_vsf_3/lazy_context_aware/0/interactions/train/epoch_300/interaction_gpu0


d:\OneDrive\Dokumente\UNI Osnabrück\bachelorthesis\InfosKristina\Code\emergent-abstractions\utils\analysis_from_interaction.py:315: RuntimeWarning: invalid value encountered in divide
  (m_entropy_concept_x_context + c_entropy_concept_x_context - joint_entropy_concept_x_context)
d:\OneDrive\Dokumente\UNI Osnabrück\bachelorthesis\InfosKristina\Code\emergent-abstractions\utils\analysis_from_interaction.py:324: RuntimeWarning: invalid value encountered in divide
  normalized_effectiveness_conc_x_cont = ((joint_entropy_concept_x_context - m_entropy_concept_x_context)
d:\OneDrive\Dokumente\UNI Osnabrück\bachelorthesis\InfosKristina\Code\emergent-abstractions\utils\analysis_from_interaction.py:331: RuntimeWarning: invalid value encountered in divide
  normalized_consistency_conc_x_cont = (


{'normalized_mutual_info': 0.849702099903696,
 'normalized_mutual_info_hierarchical': array([0.9698283 , 0.90870313, 0.85054945]),
 'normalized_mutual_info_context_dep': array([0.84266426, 0.91582143, 0.96725142]),
 'normalized_mutual_info_concept_x_context': array([0.9698283 ,        nan,        nan, 0.90578833, 0.9790615 ,
               nan, 0.86714186, 0.90961943, 0.96725142]),
 'effectiveness': 0.8617319789058988,
 'effectiveness_hierarchical': array([1.        , 0.94716929, 0.91317087]),
 'effectiveness_context_dep': array([0.78426704, 0.9089747 , 0.97858665]),
 'effectiveness_concept_x_context': array([1.        ,        nan,        nan, 0.90240457, 0.99334927,
               nan, 0.8441512 , 0.94125095, 0.97858665]),
 'consistency': 0.8380034736049569,
 'consistency_hierarchical': array([0.94142394, 0.87323939, 0.79596548]),
 'consistency_context_dep': array([0.91045775, 0.92277208, 0.95617579]),
 'consistency_concept_x_context': array([0.94142394,        nan,        nan, 0.909

##  message length

In [7]:
# we evaluated message length per hierarchy level after training but 
# you can also use the HierarchicalMessageLength callback and store the results 
# TODO: Message length results look weird, needs to be fixed!

for d in range(len(datasets)):
    
    for run in range(1): #5
        
        path_to_run = paths[d] + str(setting) +'/' + str(run) + '/' 
        interaction = load_interaction(paths[d],setting,run,n_epochs)

        attributes = n_attributes[d]
        values = n_values[d]
        scores = message_length_per_hierarchy_level(interaction, attributes)
        
        #pickle.dump(scores, open(path_to_run + 'message_length_hierarchical.pkl', 'wb'))
scores

results/(3,4)_game_size_10_vsf_3/lazy_context_aware/0/interactions/train/epoch_300/interaction_gpu0
message length tensor([1, 1, 1,  ..., 1, 1, 2])
hierarchical [1.607, 1.544, 1.53]


[1.607, 1.544, 1.53]

##  symbol redundancy

In [8]:
for d in range(len(datasets)):
    
    attributes = n_attributes[d]
    values = n_values[d]
    vs_factor = int(paths[d][-2])
    vocab_size = (n_values[d] + 1) * vs_factor + 1
    
    for run in range(1): #5
        
        path_to_run = paths[d] + str(setting) +'/' + str(run) + '/'
        #symbol_f = np.load(path_to_run + 'symbols_pernsame.npy')
        interaction = load_interaction(paths[d],setting,run,n_epochs)
        redundancy, MI = symbol_frequency(interaction, attributes, values, vocab_size)
        
        scores = {'symbol_redundancy': redundancy, 'MI_symbol-attribute_value': MI}
        
        #pickle.dump(scores, open(path_to_run + 'symbol_redundancy.pkl', 'wb'))

ValueError: invalid literal for int() with base 10: '_'

## ZLA significance scores

In [8]:
for d in range(len(datasets)):
    
    for run in range(1): # later 5

        path_to_run = paths[d] + str(setting) +'/' + str(run) + '/' 
        interaction = load_interaction(paths[d],setting,run,n_epochs)

        scores = ZLA_significance_score(interaction,num_permutations=1000)

        #pickle.dump(scores, open(path_to_run + 'ZLA_significance.pkl', 'wb'))
scores

results/(3,4)_game_size_10_vsf_3/lazy_context_aware/0/interactions/train/epoch_300/interaction_gpu0


{'mean_message_length': tensor(1.6708),
 'mean_weighted_message_length': tensor(1.5368),
 'p_zla': tensor(0.0010)}

## Symbol Informativeness

In [10]:
datasets = ['(3,4)',]
n_values = [4,]
n_attributes = [3,]
n_epochs = 300
paths = ['results/' + d + '_game_size_10_vsf_3' for d in datasets]

In [11]:
context_unaware = False # whether original or context_unaware simulations are evaluated
lazy = True # whether the lazy agent is used
impatient = False
setting = ""
if not lazy:
    start = 'impatience_' if impatient else ''
    if context_unaware:
        setting = start + 'context_unaware'
    else:
        setting = start + 'context_aware'
        if impatient == False:
            setting = 'standard'
else:
    start = 'lazimpa' if impatient else 'lazy'
    if context_unaware:
        setting = start + '_context_unaware'
    else:
        setting = start + '_context_aware'
setting

In [17]:
for d in range(len(datasets)):
    
    for run in range(1): # later 5

        path_to_run = paths[d] + str(setting) +'/' + str(run) + '/' 
        interaction = load_interaction(paths[d],setting,run,n_epochs)

        attributes = n_attributes[d]
        values = n_values[d]

        listener,_,_ = load_listener(paths[d],setting,run,attributes, values,context_unaware)

        scores = information_analysis(listener,interaction,eos_token=0.0)

        #pickle.dump(scores, open(path_to_run + 'symbol_informativeness.pkl', 'wb'))

scores

results/(3,4)_game_size_10_vsf_3/lazy_context_aware/0/interactions/train/epoch_300/interaction_gpu0
eos:  torch.Size([1807, 4])
pred:  torch.Size([1807, 4, 20])


{'positional_encoding': tensor([0.9917, 0.9364, 0.8276]),
 'effective_length': tensor(1.4909),
 'information_density': tensor(0.9701)}

In [16]:
interaction = load_interaction(paths[d],setting,run,n_epochs)
listener,_,_ = load_listener(paths[d],setting,run,attributes, values,context_unaware)
message = interaction.message
receiver_output = listener(message,interaction.receiver_input)

impatience = True

not_eosed_before = torch.ones(receiver_output.size(0))
expected_length = 0.0 # final includes eos here
z = 0.0

for step in range(receiver_output.size(1)):
    eos_mask = message[:, step, 0]  # always eos == 0
    if impatience:
        add_mask = not_eosed_before
    else:
        add_mask = eos_mask * not_eosed_before
    z += add_mask
    expected_length += add_mask.detach() * (1.0 + step)
    print(not_eosed_before)
    not_eosed_before = not_eosed_before * (1.0 - eos_mask)

expected_length += (step + 1) * not_eosed_before
z += not_eosed_before
assert z.allclose(
            z
        ), f"lost probability mass, {z.min()}, {z.max()}"
z

results/(3,4)_game_size_10_vsf_3/lazy_context_aware/0/interactions/train/epoch_300/interaction_gpu0
tensor([1., 1., 1.,  ..., 1., 1., 1.])
tensor([1., 1., 1.,  ..., 1., 1., 1.])
tensor([0.0000e+00, 0.0000e+00, 4.7684e-07,  ..., 0.0000e+00, 3.3722e-03,
        1.0000e+00])
tensor([0.0000e+00, 0.0000e+00, 4.7684e-07,  ..., 0.0000e+00, 3.3722e-03,
        0.0000e+00])


tensor([2.0000, 2.0000, 2.0000,  ..., 2.0000, 2.0067, 3.0000])

## Data analysis for attribute visualization

In [ ]:
#TODO

##  compositionality scores: topsim, posdis, bosdis

### topsim

In [ ]:
# topsim
# although topsim values are stored throughout training if callbacks are verbose, we reevaluate the
# final topsim scores with more data points 

samples = 5000 # maybe shuffle from these because otherwise I just take the first 5,000 (which might not be the best)
for d, dataset in enumerate(datasets):
    
    dim = [n_values[d]]*n_attributes[d]
    
    for run in range(5):
        print("dataset", dataset, "run", run)
        
        topsim_final = {}
        path_to_run = paths[d] + str(setting) +'/' + str(run) + '/'
        path_to_interaction_train = (path_to_run + 'interactions/train/epoch_' + str(n_epochs) + '/interaction_gpu0')
        path_to_interaction_val = (path_to_run + 'interactions/validation/epoch_' + str(n_epochs) + '/interaction_gpu0')
        
        TOPSIM = TopographicSimilarityConceptLevel(dim, is_gumbel=True)
        
        for mode in ['train', 'val']:

            if mode == 'train':
                interaction = torch.load(path_to_interaction_train)
            elif mode == 'val':
                interaction = torch.load(path_to_interaction_val)
                
                  
            messages = interaction.message.argmax(dim=-1)
            sender_input = interaction.sender_input
            n_targets = int(sender_input.shape[1]/2)
            # get target objects and fixed vectors to re-construct concepts
            target_objects = sender_input[:, :n_targets]
            target_objects = k_hot_to_attributes(target_objects, n_values[d])
            # concepts are defined by a list of target objects (here one sampled target object) and a fixed vector
            (objects, fixed) = retrieve_concepts_sampling(target_objects)
            # add one such that zero becomes an empty attribute for the calculation (_)
            objects = objects + 1
            concepts = torch.from_numpy(objects * (np.array(fixed)))
            specific_idx = np.where(np.sum(fixed, axis=1)==n_attributes[d])[0]
            messages_specific = messages[specific_idx]
            concepts_specific = concepts[specific_idx]
            
            generic_idx = np.where(np.sum(fixed, axis=1)==1)[0]
            messages_generic = messages[generic_idx]
            concepts_generic = concepts[generic_idx]

            messages = [msg.tolist() for msg in messages]
            messages_specific = [msg.tolist() for msg in messages_specific]
            messages_generic = [msg.tolist() for msg in messages_generic]

            encoded_input = encode_target_concepts_for_topsim(sender_input)
            # randomly take samples when more than 5000 samples are available
            # if len(encoded_input) > samples: 
            #     print("sampling")
            #     sample_indices = random.sample(range(len(encoded_input)), samples)
            #     sampled_input = [encoded_input[i] for i in sample_indices]
            #     sampled_messages = [messages[i] for i in sample_indices]
            #     print("start computing")
            #     print(len(sampled_input), len(sampled_input[0]), len(sampled_input[0][0]))
            #     topsim = TOPSIM.compute_topsim(sampled_input, sampled_messages)
            # else:
            topsim = TOPSIM.compute_topsim(encoded_input[0:samples], messages[0:samples]) # default: hausdorff distance for concepts, edit distance for messages
            # if len(concepts_specific) > samples:
            #     print("sampling specific")
            #     sample_indices_specific = random.sample(range(len(concepts_specific)), samples)
            #     sampled_input_specific = [concepts_specific[i] for i in sample_indices_specific]
            #     sampled_messages_specific = [messages_specific[i] for i in sample_indices_specific]
            #     topsim_specific = TOPSIM.compute_topsim(sampled_input_specific, sampled_messages_specific, 
            #                                             meaning_distance_fn="edit")
            # else:
            topsim_specific = TOPSIM.compute_topsim(concepts_specific[0:samples], messages_specific[0:samples], 
                                                        meaning_distance_fn="edit")
            
            topsim_generic = TOPSIM.compute_topsim(concepts_generic[0:samples], messages_generic[0:samples],
                                                   meaning_distance_fn="edit")

            print('... topsim computed')

            topsim_final['topsim_' + mode] = topsim
            topsim_final['topsim_specific_' + mode] = topsim_specific
            topsim_final['topsim_generic_' + mode] = topsim_generic
    
        pickle.dump(topsim_final, open(path_to_run +  "topsim_final.pkl", "wb" ) )
        print(topsim_final)        

#### Topsim over time

In [ ]:
for d, dataset in enumerate(datasets):
    
    for run in range(5):
        print("dataset", dataset, "run", run)
        
        path_to_run = paths[d] + str(setting) +'/' + str(run) + '/'
        path_to_interaction_train = (path_to_run + 'interactions/train/epoch_' + str(n_epochs) + '/interaction_gpu0')
        path_to_interaction_val = (path_to_run + 'interactions/validation/epoch_' + str(n_epochs) + '/interaction_gpu0')
        
        for mode in ['train', 'val']:

            if mode == 'train':
                interaction = torch.load(path_to_interaction_train)
            elif mode == 'val':
                interaction = torch.load(path_to_interaction_val)

        messages = interaction.message.argmax(dim=-1)
        sender_input = interaction.sender_input
        messages = [msg.tolist() for msg in messages]
        encoded_input = encode_target_concepts_for_topsim(sender_input)
        dim = [n_values[0]] * n_attributes[0]
        TOPSIM = TopographicSimilarityConceptLevel(dim, is_gumbel=True)
        
        samples = 5000
        num_batches = len(messages) // samples + (len(messages) % samples > 0)
        topsim_over_time = []
        
        for i in range(num_batches):
            messages_batch = messages[i * samples:(i + 1) * samples]
            topsim = TOPSIM.compute_topsim(encoded_input[i * samples:(i + 1) * samples], messages_batch)
            topsim_over_time.append(topsim)
            
        pickle.dump(topsim_over_time, open(path_to_run +  "topsim_over_time.pkl", "wb" ) )

### Posdis and Bosdis

In [ ]:
# use Disent callback from egg

for d in range(len(datasets)): 
    
    path = paths[d]
    dim = [n_values[d]] * n_attributes[d]
    n_features = n_attributes[d] * n_values[d]
    vs_factor = int(path[-2])
    vocab_size = (n_values[d] + 1) * vs_factor + 1
    
    print("data set", dim)
    
    for run in range(5):
        
        posdis_bosdis = {}
    
        path_to_run = paths[d] + '/' + str(setting) +'/' + str(run) + '/'
        path_to_interaction_train = (path_to_run + 'interactions/train/epoch_' + str(n_epochs) + '/interaction_gpu0')
        interaction = torch.load(path_to_interaction_train)
        
        messages = interaction.message.argmax(dim=-1)
        sender_input = interaction.sender_input
        n_targets = int(sender_input.shape[1]/2)
        # get target objects and fixed vectors to re-construct concepts
        target_objects = sender_input[:, :n_targets]
        target_objects = k_hot_to_attributes(target_objects, n_values[d])
        # concepts are defined by a list of target objects (here one sampled target object) and a fixed vector
        (objects, fixed) = retrieve_concepts_sampling(target_objects)
        # add one such that zero becomes an empty attribute for the calculation (_)
        objects = objects + 1
        concepts = torch.from_numpy(objects * (np.array(fixed)))

        # concrete/specific concepts: where all attributes are fixed
        concepts_specific = torch.tensor(
            objects[torch.sum(torch.from_numpy(fixed), dim=1) == n_attributes[d]])
        messages_specific = messages[torch.sum(torch.from_numpy(fixed), dim=1) == n_attributes[d]]

        # generic concepts: where only one attribute is fixed
        concepts_generic = torch.tensor(
            objects[torch.sum(torch.from_numpy(fixed), dim=1) == 1])
        messages_generic = messages[torch.sum(torch.from_numpy(fixed), dim=1) == 1]
        
        posdis_specific = Disent.posdis(concepts_specific, messages_specific)
        bosdis_specific = Disent.bosdis(concepts_specific, messages_specific, vocab_size)

        posdis_generic = Disent.posdis(concepts_generic, messages_generic)
        bosdis_generic = Disent.bosdis(concepts_generic, messages_generic, vocab_size)
        
        posdis = Disent.posdis(torch.from_numpy(objects), messages)
        bosdis = Disent.bosdis(torch.from_numpy(objects), messages, vocab_size)
        
        posdis_bosdis['posdis_specific'] = posdis_specific
        posdis_bosdis['bosdis_specific'] = bosdis_specific
        posdis_bosdis['posdis_generic'] = posdis_generic
        posdis_bosdis['bosdis_generic'] = bosdis_generic
        posdis_bosdis['posdis'] = posdis
        posdis_bosdis['bosdis'] = bosdis

        print(posdis_bosdis)
    
        pickle.dump(posdis_bosdis, open(path_to_run + "posdis_bosdis.pkl", "wb" ) )

#### Posdis and bosdis concept x context

In [ ]:
# bosdis concept x context
for d in range(len(datasets)):

    vs_factor = int(paths[d][-2])
    vocab_size = (n_values[d] + 1) * vs_factor + 1
    
    for run in range(5):

        path_to_run = paths[d] + '/' + str(setting) +'/' + str(run) + '/' 
        path_to_interaction = (path_to_run + 'interactions/train/epoch_' + str(n_epochs) + '/interaction_gpu0')
        interaction = torch.load(path_to_interaction)

        attributes = n_attributes[d]
        values = n_values[d]
        scores = bosdis(interaction, attributes, values, vocab_size)

        pickle.dump(scores, open(path_to_run + 'bosdis_scores.pkl', 'wb'))

In [ ]:
# posdis concept x context
for d in range(len(datasets)):

    vs_factor = int(paths[d][-2])
    vocab_size = (n_values[d] + 1) * vs_factor + 1

    for run in range(5):
        path_to_run = paths[d] + '/' + str(setting) + '/' + str(run) + '/'
        print(path_to_run)
        path_to_interaction = (path_to_run + 'interactions/train/epoch_' + str(n_epochs) + '/interaction_gpu0')
        #print(path_to_interaction)
        interaction = torch.load(path_to_interaction)

        attributes = n_attributes[d]
        values = n_values[d]
        scores = posdis(interaction, attributes, values, vocab_size)

        pickle.dump(scores, open(path_to_run + 'posdis_scores.pkl', 'wb'))

## co-occurrences

In [ ]:
# Not yet implemented:

for d in range(len(datasets)):
    
    vs_factor = int(paths[d][-2])
    
    for run in range(5): 
        
        path_to_run = paths[d] + str(setting) +'/' + str(run) + '/'
        path_to_interaction = (path_to_run + 'interactions/train/epoch_' + str(n_epochs) + '/interaction_gpu0')
        interaction = torch.load(path_to_interaction)

        attributes = n_attributes[d]
        values = n_values[d]
        
        scores = cooccurrence_per_hierarchy_level(interaction, attributes, values, vs_factor)

        print(scores)
        
        pickle.dump(scores, open(path_to_run + 'normalized_cooccurrence.pkl', 'wb'))
    